In [1]:
import undetected_chromedriver as uc
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
import os, shutil, openpyxl, csv, time
import pandas as pd

In [2]:
download_folder = 'C:/MyProjects/WebScraping/Upwork/Ahmad/Job2/downloads'
options = uc.ChromeOptions()
options.add_argument('--disable-popup-blocking')  # Optional: Disable popup blocking
options.add_experimental_option("prefs", {
    "download.default_directory": download_folder,  # Set default download directory
    "download.prompt_for_download": False,  # Disable download prompt
    "directory_upgrade": True  # Ensure the folder is created if it doesn't exist
})
driver = uc.Chrome(options=options)
driver.maximize_window()

In [ ]:
driver.get('https://training.gov.au/search?searchText=&searchType=RTO')
time.sleep(5)
driver.get('https://training.gov.au/search?searchText=&searchType=RTO&status=0&status=2&rtoTypeCode=31&rtoTypeCode=21&rtoTypeCode=25&rtoTypeCode=27&rtoTypeCode=61&rtoTypeCode=51&rtoTypeCode=53&rtoTypeCode=91&rtoTypeCode=93&rtoTypeCode=95&rtoTypeCode=97&rtoTypeCode=99&deliveredLocationState=Vic')


In [ ]:
rtos = driver.find_elements(By.CSS_SELECTOR, 'div.row.gx-3 div.col-lg-8.col-12 div.mint-card')

for rto in rtos[0:5]:
    rto_url = rto.find_element(By.CSS_SELECTOR, 'div.card-inner div.card-copy a').get_attribute('href')
    base_url = rto_url.replace('summary', '')
    
    wb = openpyxl.Workbook()
    
    
    
    #Open a new tab and load the rto_url then switch to that new tab
    driver.execute_script(f"window.open('{rto_url}', '_blank');")
    driver.switch_to.window(driver.window_handles[-1]) 
    print('Current Page: ',rto_url)
    
    
    #Contacts tab
    contacts =base_url + '/contacts'
    driver.get(contacts)
    contacts_data = {} #All data in the contacts tab is stored here.
    
    contacts_ws =wb.active 
    contacts_ws.title = 'contacts'  #Create the contacts worksheet

    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[contains(@id, "contactstab_11")]/div/div/ul')))
    contact_entries = driver.find_elements(By.XPATH, '//*[contains(@id, "contactstab_11")]/div/div/ul/li')
    #print(len(contact_entries))
    for entry in contact_entries:
        category = entry.find_element(By.XPATH, './/h2').text
        contacts_data[category] = {}
        table_rows = entry.find_elements(By.XPATH, './/table/tbody/tr')
        if table_rows:
            for row in table_rows:
                label = row.find_element(By.XPATH, './/td/strong').text
                value = row.find_element(By.XPATH, './/td[2]').text
                contacts_data[category][label] = value
        else:
            label = entry.find_element(By.XPATH, './/*[contains(@class, "row mb-1 title")]/div/strong').text.replace('s', '')
            value = entry.find_element(By.XPATH, './/*[contains(@class, "row gy-1 grid grid-2-column")]/span').text
            #print(f'{label} : {value}')
            contacts_data[category][label] = value

    unique_keys = set()
    for cat, details in contacts_data.items():
        unique_keys.update(details.keys())
    headers = ['Categories']+ list(unique_keys)
    contacts_ws.append(headers)

    for cat, details in contacts_data.items():
        row = [cat]
        for key in headers[1:]:
            row.append(details.get(key, ''))
        contacts_ws.append(row)
    
    time.sleep(3)

    #Addresses tab
    addresses =base_url + '/addresses'
    driver.get(addresses)

    addresses_ws = wb.create_sheet(title='Delivery Location')  # Create the 'Delivery Location' sheet

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[contains(@id, "table--14")]')))
        address_table = driver.find_element(By.XPATH, '//*[contains(@id, "table--14")]')
        headers = [header.text.replace('sortable\nunfold_more', '').strip() for header in address_table.find_elements(By.XPATH, './/thead/tr/th')]
        rows = address_table.find_elements(By.XPATH, './/tbody//tr')

        # Extract address data
        table_data = []
        for row in rows:
            cells = row.find_elements(By.XPATH, './/td')
            row_data = [cell.text for cell in cells]
            table_data.append(row_data)

        
        # Write the headers to the first row of the addresses sheet
        for col_num, header in enumerate(headers, 1):
            addresses_ws.cell(row=1, column=col_num, value=header)

        # Write the data to the sheet starting from row 2
        for row_num, row in enumerate(table_data, 2):
            for col_num, cell_data in enumerate(row, 1):
                addresses_ws.cell(row=row_num, column=col_num, value=cell_data)

    except Exception as e:
        pass
        #print('No address table found at:', rto_url)


# Save the workbook after populating both sheets
    wb.save(f"{rto_url.replace('https://training.gov.au/organisation/details/', '')}.xlsx")

    #Close that tab
    driver.close()



    #switch back to the main tab
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(1)
    #driver.quit()

Current Page:  https://training.gov.au/organisation/details/0022
Current Page:  https://training.gov.au/organisation/details/0049
Current Page:  https://training.gov.au/organisation/details/0058
Current Page:  https://training.gov.au/organisation/details/0080
Current Page:  https://training.gov.au/organisation/details/0095


In [6]:
driver.quit()